In [1]:
import sys
sys.path.insert(0, '/home/fangming/projects/dredfish/packages/PySpots/')

from fish_helpers import *
# import torch
# import scvi
import scanpy as sc
import anndata
sc.set_figure_params(figsize=(7, 7))

# Harmony integrate is slow to run

# Load Data

In [2]:
ref_path = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference/10X_dpnmf/'
weights = pd.read_csv(os.path.join(ref_path,'weights.csv'),index_col=0)
vizgen_distributions = pd.read_csv('/bigstore/binfo/mouse/Brain/FISH/Vizgen_data/cell_type_abundances_S2.csv',index_col=0)

In [3]:
""" Data """
""" Load Measured Data """
# data = anndata.read_h5ad('/bigstore/Images2021/gaby/dredFISH/DPNMF_PolyA_2021Nov19/results/clustered.h5ad')
measured_path = '/bigstore/Images2021/gaby/dredFISH/DPNMF_PolyA_2021Nov19/results'
if os.path.exists(os.path.join(measured_path,'clustered.h5ad')):
    measured_data = anndata.read_h5ad(os.path.join(measured_path,'clustered.h5ad'))
else:
    measured_counts = np.load(os.path.join(measured_path,'vectors.npy'),allow_pickle=True)
    measured_cell_metadata = pd.read_csv(os.path.join(measured_path,'cell_metadata.csv'),low_memory=False,index_col=0)
    measured_cells = np.array(measured_cell_metadata.index)
    measured_genes = np.array(weights.columns)
    measured_data = anndata.AnnData(X=measured_counts,var=pd.DataFrame(index=measured_genes),obs=pd.DataFrame(index=measured_cells))
    for column in measured_cell_metadata.columns:
        if measured_cell_metadata[column].dtype=='O':
            measured_data.obsm[column] = np.array(measured_cell_metadata[column].astype(str))
        else:
            measured_data.obsm[column] = np.array(measured_cell_metadata[column])
    measured_data.obs_names_make_unique()
    # measured_data.write(filename=os.path.join(measured_path,'anndata.h5ad'))
    del measured_counts,measured_cells,measured_genes,measured_cell_metadata
    
    
""" Load SmartSeq Reference """
ref_SS_path = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference/SmartSeq_dpnmf/'
if os.path.exists(os.path.join(ref_SS_path,'anndata.h5ad')):
    ref_SS_data = anndata.read_h5ad(os.path.join(ref_SS_path,'anndata.h5ad'))
else:
    ref_SS_counts = np.load(os.path.join(ref_SS_path,'projected.npy'),allow_pickle=True)
    ref_SS_cell_metadata = pd.read_csv(os.path.join(ref_SS_path,'metadata.csv'),low_memory=False,index_col=0)
    ref_SS_cells = np.array(ref_SS_cell_metadata.index)
    ref_SS_genes = np.array(weights.columns)
    ref_SS_data = anndata.AnnData(X=ref_SS_counts,var=pd.DataFrame(index=ref_SS_genes),obs=pd.DataFrame(index=ref_SS_cells))
    for column in ref_SS_cell_metadata.columns:
        if ref_SS_cell_metadata[column].dtype=='O':
            ref_SS_data.obsm[column] = np.array(ref_SS_cell_metadata[column].astype(str))
        else:
            ref_SS_data.obsm[column] = np.array(ref_SS_cell_metadata[column])
    ref_SS_data.obs_names_make_unique()
    # ref_SS_data.write(filename=os.path.join(ref_SS_path,'anndata.h5ad'))
    del ref_SS_counts,ref_SS_genes,ref_SS_cells,ref_SS_cell_metadata

In [4]:
""" Toss Cells with no signal """
measured_data = measured_data[measured_data.X.sum(1)>1]
measured_data.shape

(72370, 24)

In [5]:
measured_data

View of AnnData object with n_obs × n_vars = 72370 × 24
    obs: 'label', 'pixel_x', 'pixel_y', 'nuclei_size', 'nuclei_signal', 'cytoplasm_size', 'cytoplasm_signal', 'total_size', 'total_signal', 'posname', 'posname_stage_x', 'posname_stage_y', 'stage_x', 'stage_y', 'leiden', 'opt_types'
    uns: 'leiden', 'neighbors'
    obsm: 'stage'
    layers: 'cytoplasm_vectors', 'nuclei_vectors', 'total_vectors'
    obsp: 'connectivities', 'distances'

In [6]:
""" right side only """
xy = np.zeros([measured_data.shape[0],2])
xy[:,0] = measured_data.obs['stage_x']
xy[:,1] = -measured_data.obs['stage_y']
measured_data.obsm['stage'] = xy
# measured_data = measured_data[measured_data.obsm['stage'][:,0]>1000]

In [7]:
""" Class Balance to Vizgen Distributions """
total_cells = measured_data.shape[0]
ref_SS_data_balanced_idx = []
labels = ref_SS_data.obsm['Level_3_subclass_label'].astype(str)
for label in vizgen_distributions.index:
    idxes = np.where(labels==label)[0]
    t_ncells = int(total_cells*vizgen_distributions.loc[label]['0'])
    if idxes.shape[0]<t_ncells:
        idxes = np.random.choice(idxes,t_ncells,replace=True)
    else:
        idxes = np.random.choice(idxes,t_ncells,replace=False)
    ref_SS_data_balanced_idx.extend(list(idxes))
ref_SS_data_balanced = ref_SS_data[ref_SS_data_balanced_idx,:]
print(ref_SS_data_balanced.shape)

(72348, 24)


/tmp/ipykernel_3781438/3854629698.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  idxes = np.where(labels==label)[0]


In [8]:
measured_data.layers['DPNMF'] = measured_data.X.copy()
ref_SS_data_balanced.layers['DPNMF'] = ref_SS_data_balanced.X.copy()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [9]:
data = measured_data
reference_data = ref_SS_data_balanced

In [10]:
Levels = [i for i in reference_data.obsm if i.split('_')[0]=='Level']
Levels

['Level_1_class_label',
 'Level_2_neighborhood_label',
 'Level_3_subclass_label',
 'Level_4_supertype_label',
 'Level_5_cluster_label']

# Level 1 Harmonization

In [11]:
temp_data = data
temp_ref_data = reference_data

In [12]:
""" Bitwise Correction Factor """
""" Move to Same Space """
""" Move Vectors to same space (Relative) """
from sklearn.preprocessing import normalize
def normalize_vector(v,thresh = 25):
    v = v.copy()
    v = v-np.percentile(v,50)
    v = v/(np.percentile(v,75)-np.percentile(v,25))
    return v

norm_temp_data = np.array(temp_data.layers['DPNMF'].copy())
norm_temp_ref_data = np.array(temp_ref_data.layers['DPNMF'].copy())
for i in range(norm_temp_data.shape[1]):
    """ Scale bitwise first """
    norm_temp_data[:,i] = normalize_vector(norm_temp_data[:,i])
    norm_temp_ref_data[:,i] = normalize_vector(norm_temp_ref_data[:,i])
temp_data.layers['DPNMF_IQR'] = norm_temp_data.copy()
temp_ref_data.layers['DPNMF_IQR'] = norm_temp_ref_data.copy()

In [13]:
temp_data = temp_data[temp_data.layers['DPNMF_IQR'].max(1)!=0]
temp_ref_data = temp_ref_data[temp_ref_data.layers['DPNMF_IQR'].max(1)!=0]

In [14]:
""" Try Harmony Anchor Reference"""
merged_data = anndata.AnnData(X=np.concatenate([temp_data.layers['DPNMF_IQR'].astype(float),temp_ref_data.layers['DPNMF_IQR'].astype(float)]),
                              var=temp_data.var,
                              obs=pd.DataFrame(np.concatenate([np.array(temp_data.obs.index),np.array(temp_ref_data.obs.index)])))
merged_data.obs['dataset'] = np.concatenate([np.array(['a' for i in range(len(temp_data.obs))]),np.array(['b' for i in range(len(temp_ref_data.obs))])])
merged_data.obs['label'] = np.concatenate([np.array(['unknown' for i in range(len(temp_data.obs))]),np.array(temp_ref_data.obsm['Level_3_subclass_label'].astype(str))])
merged_data.obsm['DPNMF'] = merged_data.X.copy()
merged_data.layers['DPNMF'] = merged_data.X.copy()
merged_data.raw = merged_data

import scanpy.external as sce
sce.pp.harmony_integrate(merged_data, 'dataset',basis='DPNMF',adjusted_basis='DPNMF_harmony',reference_values='b')

temp_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'a'].obsm['DPNMF_harmony'])
temp_ref_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'b'].obsm['DPNMF_harmony'])

/home/fangming/anaconda3/envs/routine/lib/python3.10/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


ValueError: operands could not be broadcast together with shapes (98,144718) (100,1) 

In [ ]:
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_IQR']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pl.pca(temp_data)
sc.pl.pca_variance_ratio(temp_data, log=False)

""" Recluster """
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(temp_data, init_pos='paga')
sc.pl.umap(temp_data, color=['leiden'])
plt.figure(figsize=[25,15])
sc.pl.embedding(temp_data,'stage', color=['leiden'])

In [ ]:
""" Check Correlation with ss Data """
from scipy.stats import pearsonr,spearmanr
temp_ref_data.obs['label'] = temp_ref_data.obsm['Level_1_class_label']
ref_cell_types = np.unique(temp_ref_data.obs['label'])
measured_cell_types = np.unique(temp_data.obs['leiden']) 
corr_matrix = np.zeros([ref_cell_types.shape[0],measured_cell_types.shape[0]])
for i,r in enumerate(ref_cell_types):
    r_v = temp_ref_data[temp_ref_data.obs['label']==r].obsm['DPNMF_harmony'].mean(0)
    for j,m in enumerate(measured_cell_types):
        m_v = temp_data[temp_data.obs['leiden']==m].obsm['DPNMF_harmony'].mean(0)
        corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
        # corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
corr_df = pd.DataFrame(corr_matrix,index=ref_cell_types,columns=measured_cell_types)
plt.figure(figsize=[10,10])
sns.clustermap(corr_df,cmap='bwr',center=0)
plt.title('Correlation')
plt.show()
""" Use These as label transfer """
converter = {measured_cell_types[i]:j for i,j in enumerate(np.array(corr_df.index)[np.array(corr_df).argmax(0)])}
transfered = np.array([converter[i] for i in temp_data.obs['leiden']])
temp_data.obs['Level_1_class_label_predicted'] = transfered
temp_data.obsm['Level_1_class_label_predicted'] = transfered
sc.pl.embedding(temp_data,'stage', color=['leiden','Level_1_class_label_predicted'])

In [ ]:
data.obsm['Level_1_class_label_predicted'] = np.array(['unknown' for i in range(data.shape[0])])
data.obs['Level_1_class_label_predicted'] = np.array(['unknown' for i in range(data.shape[0])])

temp = pd.DataFrame(data.obs['Level_1_class_label_predicted'])
temp.loc[temp_data.obs.index] = np.array(temp_data.obs['Level_1_class_label_predicted'])[:,None]
data.obs['Level_1_class_label_predicted'] = np.array(temp)
data.obsm['Level_1_class_label_predicted'] = np.array(data.obs['Level_1_class_label_predicted'])

# Level 2 Cell Type Harmonize

## Setup

In [ ]:
data.obsm['Level_2_neighborhood_label_predicted'] = np.array(['unknown' for i in range(data.shape[0])])
data.obs['Level_2_neighborhood_label_predicted'] = np.array(['unknown' for i in range(data.shape[0])])

## Glutamatergic

In [ ]:
Label = 'Glutamatergic'
temp_data = data[data.obsm['Level_1_class_label_predicted']==Label]
temp_ref_data = reference_data[reference_data.obsm['Level_1_class_label']==Label]

In [ ]:
""" Bitwise Correction Factor """
""" Move to Same Space """
""" Move Vectors to same space (Relative) """
from sklearn.preprocessing import normalize
def normalize_vector(v,thresh = 25):
    v = v.copy()
    v = v-np.percentile(v,50)
    v = v/(np.percentile(v,75)-np.percentile(v,25))
    return v

norm_temp_data = np.array(temp_data.layers['DPNMF'].copy())
norm_temp_ref_data = np.array(temp_ref_data.layers['DPNMF'].copy())
for i in range(norm_temp_data.shape[1]):
    """ Scale bitwise first """
    norm_temp_data[:,i] = normalize_vector(norm_temp_data[:,i])
    norm_temp_ref_data[:,i] = normalize_vector(norm_temp_ref_data[:,i])
temp_data.layers['DPNMF_IQR'] = norm_temp_data.copy()
temp_ref_data.layers['DPNMF_IQR'] = norm_temp_ref_data.copy()

In [ ]:
temp_data = temp_data[temp_data.layers['DPNMF_IQR'].max(1)!=0]
temp_ref_data = temp_ref_data[temp_ref_data.layers['DPNMF_IQR'].max(1)!=0]

In [ ]:
""" Try Harmony Anchor Reference"""
merged_data = anndata.AnnData(X=np.concatenate([temp_data.layers['DPNMF_IQR'].astype(float),temp_ref_data.layers['DPNMF_IQR'].astype(float)]),
                              var=temp_data.var,
                              obs=pd.DataFrame(np.concatenate([np.array(temp_data.obs.index),np.array(temp_ref_data.obs.index)])))
merged_data.obs['dataset'] = np.concatenate([np.array(['a' for i in range(len(temp_data.obs))]),np.array(['b' for i in range(len(temp_ref_data.obs))])])
merged_data.obs['label'] = np.concatenate([np.array(['unknown' for i in range(len(temp_data.obs))]),np.array(temp_ref_data.obsm['Level_3_subclass_label'].astype(str))])
merged_data.obsm['DPNMF'] = merged_data.X.copy()
merged_data.layers['DPNMF'] = merged_data.X.copy()
merged_data.raw = merged_data

import scanpy.external as sce
sce.pp.harmony_integrate(merged_data, 'dataset',basis='DPNMF',adjusted_basis='DPNMF_harmony',reference_values='b')

temp_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'a'].obsm['DPNMF_harmony'])
temp_ref_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'b'].obsm['DPNMF_harmony'])

In [ ]:
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_IQR']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pl.pca(temp_data)
sc.pl.pca_variance_ratio(temp_data, log=False)

""" Recluster """
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(temp_data, init_pos='paga')
sc.pl.umap(temp_data, color=['leiden'])
plt.figure(figsize=[25,15])
sc.pl.embedding(temp_data,'stage', color=['leiden'])

In [ ]:
""" Check Correlation with ss Data """
from scipy.stats import pearsonr,spearmanr
temp_ref_data.obs['label'] = temp_ref_data.obsm['Level_2_neighborhood_label']
ref_cell_types = np.unique(temp_ref_data.obs['label'])
measured_cell_types = np.unique(temp_data.obs['leiden']) 
corr_matrix = np.zeros([ref_cell_types.shape[0],measured_cell_types.shape[0]])
for i,r in enumerate(ref_cell_types):
    r_v = temp_ref_data[temp_ref_data.obs['label']==r].obsm['DPNMF_harmony'].mean(0)
    for j,m in enumerate(measured_cell_types):
        m_v = temp_data[temp_data.obs['leiden']==m].obsm['DPNMF_harmony'].mean(0)
        corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
        # corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
corr_df = pd.DataFrame(corr_matrix,index=ref_cell_types,columns=measured_cell_types)
plt.figure(figsize=[10,10])
sns.clustermap(corr_df,cmap='bwr',center=0)
plt.title('Correlation')
plt.show()
""" Use These as label transfer """
converter = {measured_cell_types[i]:j for i,j in enumerate(np.array(corr_df.index)[np.array(corr_df).argmax(0)])}
transfered = np.array([converter[i] for i in temp_data.obs['leiden']])
temp_data.obs['Level_2_neighborhood_label_predicted'] = transfered
temp_data.obsm['Level_2_neighborhood_label_predicted'] = transfered
sc.pl.embedding(temp_data,'stage', color=['leiden','Level_2_neighborhood_label_predicted'])

In [ ]:
temp = pd.DataFrame(data.obs['Level_2_neighborhood_label_predicted'])
temp.loc[temp_data.obs.index] = np.array(temp_data.obs['Level_2_neighborhood_label_predicted'])[:,None]
data.obs['Level_2_neighborhood_label_predicted'] = np.array(temp)
data.obsm['Level_2_neighborhood_label_predicted'] = np.array(data.obs['Level_2_neighborhood_label_predicted'])

## GABAergic

In [ ]:
Label = 'GABAergic'
temp_data = data[data.obsm['Level_1_class_label_predicted']==Label]
temp_ref_data = reference_data[reference_data.obsm['Level_1_class_label']==Label]

In [ ]:
""" Bitwise Correction Factor """
""" Move to Same Space """
""" Move Vectors to same space (Relative) """
from sklearn.preprocessing import normalize
def normalize_vector(v,thresh = 25):
    v = v.copy()
    v = v-np.percentile(v,50)
    v = v/(np.percentile(v,75)-np.percentile(v,25))
    return v

norm_temp_data = np.array(temp_data.layers['DPNMF'].copy())
norm_temp_ref_data = np.array(temp_ref_data.layers['DPNMF'].copy())
for i in range(norm_temp_data.shape[1]):
    """ Scale bitwise first """
    norm_temp_data[:,i] = normalize_vector(norm_temp_data[:,i])
    norm_temp_ref_data[:,i] = normalize_vector(norm_temp_ref_data[:,i])
temp_data.layers['DPNMF_IQR'] = norm_temp_data.copy()
temp_ref_data.layers['DPNMF_IQR'] = norm_temp_ref_data.copy()

In [ ]:
temp_data = temp_data[temp_data.layers['DPNMF_IQR'].max(1)!=0]
temp_ref_data = temp_ref_data[temp_ref_data.layers['DPNMF_IQR'].max(1)!=0]

In [ ]:
""" Try Harmony Anchor Reference"""
merged_data = anndata.AnnData(X=np.concatenate([temp_data.layers['DPNMF_IQR'].astype(float),temp_ref_data.layers['DPNMF_IQR'].astype(float)]),
                              var=temp_data.var,
                              obs=pd.DataFrame(np.concatenate([np.array(temp_data.obs.index),np.array(temp_ref_data.obs.index)])))
merged_data.obs['dataset'] = np.concatenate([np.array(['a' for i in range(len(temp_data.obs))]),np.array(['b' for i in range(len(temp_ref_data.obs))])])
merged_data.obs['label'] = np.concatenate([np.array(['unknown' for i in range(len(temp_data.obs))]),np.array(temp_ref_data.obsm['Level_3_subclass_label'].astype(str))])
merged_data.obsm['DPNMF'] = merged_data.X.copy()
merged_data.layers['DPNMF'] = merged_data.X.copy()
merged_data.raw = merged_data

import scanpy.external as sce
sce.pp.harmony_integrate(merged_data, 'dataset',basis='DPNMF',adjusted_basis='DPNMF_harmony',reference_values='b')

temp_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'a'].obsm['DPNMF_harmony'])
temp_ref_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'b'].obsm['DPNMF_harmony'])

In [ ]:
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_IQR']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pl.pca(temp_data)
sc.pl.pca_variance_ratio(temp_data, log=False)

""" Recluster """
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(temp_data, init_pos='paga')
sc.pl.umap(temp_data, color=['leiden'])
plt.figure(figsize=[25,15])
sc.pl.embedding(temp_data,'stage', color=['leiden'])

In [ ]:
""" Check Correlation with ss Data """
from scipy.stats import pearsonr,spearmanr
temp_ref_data.obs['label'] = temp_ref_data.obsm['Level_2_neighborhood_label']
ref_cell_types = np.unique(temp_ref_data.obs['label'])
measured_cell_types = np.unique(temp_data.obs['leiden']) 
corr_matrix = np.zeros([ref_cell_types.shape[0],measured_cell_types.shape[0]])
for i,r in enumerate(ref_cell_types):
    r_v = temp_ref_data[temp_ref_data.obs['label']==r].obsm['DPNMF_harmony'].mean(0)
    for j,m in enumerate(measured_cell_types):
        m_v = temp_data[temp_data.obs['leiden']==m].obsm['DPNMF_harmony'].mean(0)
        corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
        # corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
corr_df = pd.DataFrame(corr_matrix,index=ref_cell_types,columns=measured_cell_types)
plt.figure(figsize=[10,10])
sns.clustermap(corr_df,cmap='bwr',center=0)
plt.title('Correlation')
plt.show()
""" Use These as label transfer """
converter = {measured_cell_types[i]:j for i,j in enumerate(np.array(corr_df.index)[np.array(corr_df).argmax(0)])}
transfered = np.array([converter[i] for i in temp_data.obs['leiden']])
temp_data.obs['Level_2_neighborhood_label_predicted'] = transfered
temp_data.obsm['Level_2_neighborhood_label_predicted'] = transfered
sc.pl.embedding(temp_data,'stage', color=['leiden','Level_2_neighborhood_label_predicted'])

In [ ]:
temp = pd.DataFrame(data.obs['Level_2_neighborhood_label_predicted'])
temp.loc[temp_data.obs.index] = np.array(temp_data.obs['Level_2_neighborhood_label_predicted'])[:,None]
data.obs['Level_2_neighborhood_label_predicted'] = np.array(temp)
data.obsm['Level_2_neighborhood_label_predicted'] = np.array(data.obs['Level_2_neighborhood_label_predicted'])

## Non Neuronal

In [ ]:
""" Keeping 'Astro/Olig' and 'Immune/Vascular' together for now """

In [ ]:
Label = ['Astro/Olig','Immune/Vascular']
temp_data = data[np.isin(data.obsm['Level_1_class_label_predicted'],Label)]
temp_ref_data = reference_data[np.isin(reference_data.obsm['Level_1_class_label'],Label)]

In [ ]:
""" Bitwise Correction Factor """
""" Move to Same Space """
""" Move Vectors to same space (Relative) """
from sklearn.preprocessing import normalize
def normalize_vector(v,thresh = 25):
    v = v.copy()
    v = v-np.percentile(v,50)
    v = v/(np.percentile(v,75)-np.percentile(v,25))
    return v

norm_temp_data = np.array(temp_data.layers['DPNMF'].copy())
norm_temp_ref_data = np.array(temp_ref_data.layers['DPNMF'].copy())
for i in range(norm_temp_data.shape[1]):
    """ Scale bitwise first """
    norm_temp_data[:,i] = normalize_vector(norm_temp_data[:,i])
    norm_temp_ref_data[:,i] = normalize_vector(norm_temp_ref_data[:,i])
temp_data.layers['DPNMF_IQR'] = norm_temp_data.copy()
temp_ref_data.layers['DPNMF_IQR'] = norm_temp_ref_data.copy()

In [ ]:
temp_data = temp_data[temp_data.layers['DPNMF_IQR'].max(1)!=0]
temp_ref_data = temp_ref_data[temp_ref_data.layers['DPNMF_IQR'].max(1)!=0]

In [ ]:
temp_data.shape,temp_ref_data.shape

In [ ]:
""" Try Harmony Anchor Reference"""
merged_data = anndata.AnnData(X=np.concatenate([temp_data.layers['DPNMF_IQR'].astype(float),temp_ref_data.layers['DPNMF_IQR'].astype(float)]),
                              var=temp_data.var,
                              obs=pd.DataFrame(np.concatenate([np.array(temp_data.obs.index),np.array(temp_ref_data.obs.index)])))
merged_data.obs['dataset'] = np.concatenate([np.array(['a' for i in range(len(temp_data.obs))]),np.array(['b' for i in range(len(temp_ref_data.obs))])])
merged_data.obs['label'] = np.concatenate([np.array(['unknown' for i in range(len(temp_data.obs))]),np.array(temp_ref_data.obsm['Level_3_subclass_label'].astype(str))])
merged_data.obsm['DPNMF'] = merged_data.X.copy()
merged_data.layers['DPNMF'] = merged_data.X.copy()
merged_data.raw = merged_data

import scanpy.external as sce
sce.pp.harmony_integrate(merged_data, 'dataset',basis='DPNMF',adjusted_basis='DPNMF_harmony',reference_values='b')

temp_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'a'].obsm['DPNMF_harmony'])
temp_ref_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'b'].obsm['DPNMF_harmony'])

In [ ]:
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_IQR']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pl.pca(temp_data)
sc.pl.pca_variance_ratio(temp_data, log=False)

""" Recluster """
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(temp_data, init_pos='paga')
sc.pl.umap(temp_data, color=['leiden'])
plt.figure(figsize=[25,15])
sc.pl.embedding(temp_data,'stage', color=['leiden'])

In [ ]:
""" Check Correlation with ss Data """
from scipy.stats import pearsonr,spearmanr
temp_ref_data.obs['label'] = temp_ref_data.obsm['Level_2_neighborhood_label']
ref_cell_types = np.unique(temp_ref_data.obs['label'])
measured_cell_types = np.unique(temp_data.obs['leiden']) 
corr_matrix = np.zeros([ref_cell_types.shape[0],measured_cell_types.shape[0]])
for i,r in enumerate(ref_cell_types):
    r_v = temp_ref_data[temp_ref_data.obs['label']==r].obsm['DPNMF_harmony'].mean(0)
    for j,m in enumerate(measured_cell_types):
        m_v = temp_data[temp_data.obs['leiden']==m].obsm['DPNMF_harmony'].mean(0)
        corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
        # corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
corr_df = pd.DataFrame(corr_matrix,index=ref_cell_types,columns=measured_cell_types)
plt.figure(figsize=[10,10])
sns.clustermap(corr_df,cmap='bwr',center=0,row_cluster=False)
plt.title('Correlation')
plt.show()
""" Use These as label transfer """
converter = {measured_cell_types[i]:j for i,j in enumerate(np.array(corr_df.index)[np.array(corr_df).argmax(0)])}
transfered = np.array([converter[i] for i in temp_data.obs['leiden']])
temp_data.obs['Level_2_neighborhood_label_predicted'] = transfered
temp_data.obsm['Level_2_neighborhood_label_predicted'] = transfered
sc.pl.embedding(temp_data,'stage', color=['leiden','Level_2_neighborhood_label_predicted'])

In [ ]:
temp = pd.DataFrame(data.obs['Level_2_neighborhood_label_predicted'])
temp.loc[temp_data.obs.index] = np.array(temp_data.obs['Level_2_neighborhood_label_predicted'])[:,None]
data.obs['Level_2_neighborhood_label_predicted'] = np.array(temp)
data.obsm['Level_2_neighborhood_label_predicted'] = np.array(data.obs['Level_2_neighborhood_label_predicted'])

# Confusion matrix

In [ ]:
temp = np.array([i+'_Other' for i in data.obs['Level_1_class_label_predicted']])
temp2 = np.array(data.obs['Level_2_neighborhood_label_predicted'])
temp2[temp2=='Other'] = temp[temp2=='Other']
data.obs['Level_2_neighborhood_label_predicted_updated'] = np.array(temp2)
data.obsm['Level_2_neighborhood_label_predicted_updated'] = np.array(data.obs['Level_2_neighborhood_label_predicted_updated'])
sc.pl.embedding(data,'stage', color=['leiden','Level_2_neighborhood_label_predicted_updated'])

In [ ]:
temp_data = data
temp_ref_data = ref_SS_data

In [ ]:
data_labels = np.array(temp_data.obs['Level_2_neighborhood_label_predicted_updated'])
ref_data_labels = np.array(temp_data.obs['opt_types'])
data_labels_unique = np.unique(data_labels)
ref_data_labels_unique = np.unique(ref_data_labels)
cmat = np.zeros([data_labels_unique.shape[0],ref_data_labels_unique.shape[0]])
for i,i_label in tqdm(enumerate(data_labels_unique),total=data_labels_unique.shape[0]):
    i_mask =data_labels==i_label
    for j,j_label in enumerate(ref_data_labels_unique):
        j_mask = ref_data_labels==j_label
        cmat[i,j] = np.sum(i_mask&j_mask)
cmat = cmat/cmat.sum(1)[:,None]
cmat = pd.DataFrame(cmat,index=data_labels_unique,columns=ref_data_labels_unique)
plt.figure(figsize=[15,15])
p = sns.clustermap(cmat.T,cmap='jet',vmin=0,vmax=1)

## L2 Result

In [ ]:
""" L2 has shared labels across different L1 populations """

In [ ]:
sc.pl.embedding(data,'stage', color=['Level_1_class_label_predicted','Level_2_neighborhood_label_predicted'])

In [ ]:
for ct in np.unique(data.obsm['Level_1_class_label_predicted']):
    m1 = data.obsm['Level_1_class_label_predicted']==ct
    m2 = data.obsm['Level_2_neighborhood_label_predicted']=='Other'
    data.obsm['Level_2_neighborhood_label_predicted'][m1&m2] = ct+'_Other'
    # data.obs['Level_2_neighborhood_label_predicted'][m1&m2] = ct+'_Other'

In [ ]:
data.obs['Level_2_neighborhood_label_predicted'] = data.obsm['Level_2_neighborhood_label_predicted']

In [ ]:
sc.pl.embedding(data,'stage', color=['Level_1_class_label_predicted','Level_2_neighborhood_label_predicted'])

## Show L2 Subclusters without calling types

In [ ]:
np.unique(data.obsm['Level_2_neighborhood_label_predicted'])

In [ ]:
data.obsm['Level_2_subclusters'] = np.array(['unknown' for i in range(data.shape[0])])
data.obs['Level_2_subclusters'] = np.array(['unknown' for i in range(data.shape[0])])

In [ ]:
for ct in tqdm(np.unique(data.obsm['Level_2_neighborhood_label_predicted'])):
    temp_data = data[np.isin(data.obsm['Level_2_neighborhood_label_predicted'],ct)]
    """ Recluster """
    temp_data.X = temp_data.layers['DPNMF_IQR']
    sc.tl.pca(temp_data, svd_solver='arpack')
    # sc.pl.pca(temp_data)
    # sc.pl.pca_variance_ratio(temp_data, log=False)

    """ Recluster """
    sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
    sc.tl.leiden(temp_data)
    sc.tl.paga(temp_data)
    sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
    sc.tl.umap(temp_data, init_pos='paga')
    # sc.pl.umap(temp_data, color=['leiden'])
    # plt.figure(figsize=[25,15])
    # sc.pl.embedding(temp_data,'stage', color=['leiden'])
    
    temp = pd.DataFrame(data.obs['Level_2_subclusters'])
    temp.loc[temp_data.obs.index] = np.array(temp_data.obs['Level_2_subclusters'])[:,None]
    temp.loc[temp_data.obs.index] = np.array([ct+'_cluster_'+str(l) for l in temp_data.obs['leiden']])[:,None]
    data.obs['Level_2_subclusters'] = np.array(temp)
    data.obsm['Level_2_subclusters'] = np.array(data.obs['Level_2_subclusters'])
    

In [ ]:
from matplotlib import gridspec
def plot_custom(temp_data,basis,marker_size=1):
    fig = plt.figure()
    fig.set_figheight(7.5)
    fig.set_figwidth(25)
    spec = gridspec.GridSpec(ncols=3, nrows=1,
                             width_ratios=[5,5,1], wspace=0.1,
                             hspace=0.5)
    ax1 = fig.add_subplot(spec[0])
    ax2 = fig.add_subplot(spec[1])
    ax3 = fig.add_subplot(spec[2])
    ax1.set_title('Stage',fontweight="bold", size=20)
    ax2.set_title('UMAP',fontweight="bold", size=20)
    # ax1.title.set_text('Stage')
    # ax2.title.set_text('UMAP')
    fig.suptitle(basis)
    for c in np.unique(temp_data.obsm[basis]):
        cm = temp_data.obsm[basis]==c
        ax1.scatter(temp_data[cm].obsm['stage'][:,0],temp_data[cm].obsm['stage'][:,1],s=marker_size,label=c)
        ax2.scatter(temp_data[cm].obsm['X_umap'][:,0],temp_data[cm].obsm['X_umap'][:,1],s=marker_size,label=c)
    handles, labels = ax2.get_legend_handles_labels()
    ax3.legend(handles, labels, loc='center', fontsize=20, markerscale=5)
    ax3.axis('off')
    ax1.grid(False)
    ax2.grid(False)
    ax3.grid(False)
    plt.grid(False)
    plt.show()
# plot_custom(temp_data,'Level_1_class_label_predicted')

In [ ]:
plot_custom(temp_data,'Level_1_class_label_predicted')

In [ ]:
temp_data = data
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_IQR']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)
sc.tl.umap(temp_data, init_pos='paga')
plot_custom(temp_data,'Level_1_class_label_predicted')

for L1 in np.unique(data.obsm['Level_1_class_label_predicted']):
    if L1=='unknown':
        continue
    M1 = data.obsm['Level_1_class_label_predicted']==L1
    temp_data = data[M1]
    """ Recluster """
    temp_data.X = temp_data.layers['DPNMF_IQR']
    sc.tl.pca(temp_data, svd_solver='arpack')
    sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
    sc.tl.leiden(temp_data)
    sc.tl.paga(temp_data)
    sc.pl.paga(temp_data, plot=False)
    sc.tl.umap(temp_data, init_pos='paga')
    plot_custom(temp_data,'Level_2_neighborhood_label_predicted')
    
    for L2 in np.unique(data[M1].obsm['Level_2_neighborhood_label_predicted']):
        if 'unknown' in L2:
            continue
        M2 = M1&(data.obsm['Level_2_neighborhood_label_predicted']==L2)
        temp_data = data[M2]
        """ Recluster """
        temp_data.X = temp_data.layers['DPNMF_IQR']
        sc.tl.pca(temp_data, svd_solver='arpack')
        sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
        sc.tl.leiden(temp_data)
        sc.tl.paga(temp_data)
        sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
        sc.tl.umap(temp_data, init_pos='paga')
        plot_custom(temp_data,'Level_2_subclusters')

In [ ]:
from sklearn.preprocessing import normalize
data.layers['DPNMF_L2'] = normalize(data.layers['DPNMF_IQR'])
temp_data = data
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_L2']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)
sc.tl.umap(temp_data, init_pos='paga')
plot_custom(temp_data,'Level_1_class_label_predicted')

for L1 in np.unique(data.obsm['Level_1_class_label_predicted']):
    if L1=='unknown':
        continue
    M1 = data.obsm['Level_1_class_label_predicted']==L1
    temp_data = data[M1]
    """ Recluster """
    temp_data.X = temp_data.layers['DPNMF_L2']
    sc.tl.pca(temp_data, svd_solver='arpack')
    sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
    sc.tl.leiden(temp_data)
    sc.tl.paga(temp_data)
    sc.pl.paga(temp_data, plot=False)
    sc.tl.umap(temp_data, init_pos='paga')
    plot_custom(temp_data,'Level_2_neighborhood_label_predicted')
    
    for L2 in np.unique(data[M1].obsm['Level_2_neighborhood_label_predicted']):
        if 'unknown' in L2:
            continue
        M2 = M1&(data.obsm['Level_2_neighborhood_label_predicted']==L2)
        temp_data = data[M2]
        """ Recluster """
        temp_data.X = temp_data.layers['DPNMF_L2']
        sc.tl.pca(temp_data, svd_solver='arpack')
        sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
        sc.tl.leiden(temp_data)
        sc.tl.paga(temp_data)
        sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
        sc.tl.umap(temp_data, init_pos='paga')
        plot_custom(temp_data,'Level_2_subclusters')

In [ ]:
from sklearn.preprocessing import normalize
plt.style.use('dark_background')
data.layers['DPNMF_L2'] = normalize(data.layers['DPNMF_IQR'])
temp_data = data
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_L2']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)
sc.tl.umap(temp_data, init_pos='paga')
plot_custom(temp_data,'Level_1_class_label_predicted')

for L1 in np.unique(data.obsm['Level_1_class_label_predicted']):
    if L1=='unknown':
        continue
    M1 = data.obsm['Level_1_class_label_predicted']==L1
    temp_data = data[M1]
    """ Recluster """
    temp_data.X = temp_data.layers['DPNMF_L2']
    sc.tl.pca(temp_data, svd_solver='arpack')
    sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
    sc.tl.leiden(temp_data)
    sc.tl.paga(temp_data)
    sc.pl.paga(temp_data, plot=False)
    sc.tl.umap(temp_data, init_pos='paga')
    plot_custom(temp_data,'Level_2_neighborhood_label_predicted')
    
    for L2 in np.unique(data[M1].obsm['Level_2_neighborhood_label_predicted']):
        if 'unknown' in L2:
            continue
        M2 = M1&(data.obsm['Level_2_neighborhood_label_predicted']==L2)
        temp_data = data[M2]
        """ Recluster """
        temp_data.X = temp_data.layers['DPNMF_L2']
        sc.tl.pca(temp_data, svd_solver='arpack')
        sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
        sc.tl.leiden(temp_data)
        sc.tl.paga(temp_data)
        sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
        sc.tl.umap(temp_data, init_pos='paga')
        plot_custom(temp_data,'Level_2_subclusters')

In [ ]:
L1 = 'Glutamatergic'
M1 = data.obsm['Level_1_class_label_predicted']==L1
L2 = 'DG_SUB_CA'
M2 = M1&(data.obsm['Level_2_neighborhood_label_predicted']==L2)

temp_data = data[M2]
temp_data = temp_data[temp_data.obsm['stage'][:,1]>0]
temp_data = temp_data[temp_data.obsm['stage'][:,1]<1600]
temp_data = temp_data[temp_data.obsm['stage'][:,0]<3000]
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_L2']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(temp_data, init_pos='paga')
plot_custom(temp_data,'Level_2_subclusters',marker_size=5)

In [ ]:
L1 = 'Glutamatergic'
M1 = data.obsm['Level_1_class_label_predicted']==L1
L2 = 'DG_SUB_CA'
M2 = M1&(data.obsm['Level_2_neighborhood_label_predicted']==L2)

temp_data = data[M2]
temp_data = temp_data[temp_data.obsm['stage'][:,1]>0]
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_L2']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(temp_data, init_pos='paga')
plot_custom(temp_data,'Level_2_subclusters',marker_size=5)

# Level 3 Clustering

In [ ]:
np.unique(data.obsm['Level_1_class_label_predicted'])

In [ ]:
np.unique(data.obsm['Level_2_neighborhood_label_predicted'])

In [ ]:
data.obsm['Level_3_subclass_label_predicted'] = np.array(['unknown' for i in range(data.shape[0])])
data.obs['Level_3_subclass_label_predicted'] = np.array(['unknown' for i in range(data.shape[0])])

## Glutamatergic

In [ ]:
Label = 'Glutamatergic'
np.unique(data[data.obsm['Level_1_class_label_predicted']==Label].obsm['Level_2_neighborhood_label_predicted'])

##### DG_SUB_CA

In [ ]:
Label1 = ['Glutamatergic']
Label2 = ['DG_SUB_CA']
temp_data = data[(np.isin(data.obsm['Level_1_class_label_predicted'],Label1)&np.isin(data.obsm['Level_2_neighborhood_label_predicted'],Label2))]
temp_ref_data = reference_data[(np.isin(reference_data.obsm['Level_1_class_label'],Label1)&np.isin(reference_data.obsm['Level_2_neighborhood_label'],Label2))]
temp_data.shape,temp_ref_data.shape

In [ ]:
""" Bitwise Correction Factor """
""" Move to Same Space """
""" Move Vectors to same space (Relative) """
from sklearn.preprocessing import normalize
def normalize_vector(v,thresh = 25):
    v = v.copy()
    v = v-np.percentile(v,50)
    v = v/(np.percentile(v,75)-np.percentile(v,25))
    return v

norm_temp_data = np.array(temp_data.layers['DPNMF'].copy())
norm_temp_ref_data = np.array(temp_ref_data.layers['DPNMF'].copy())
for i in range(norm_temp_data.shape[1]):
    """ Scale bitwise first """
    norm_temp_data[:,i] = normalize_vector(norm_temp_data[:,i])
    norm_temp_ref_data[:,i] = normalize_vector(norm_temp_ref_data[:,i])
temp_data.layers['DPNMF_IQR'] = norm_temp_data.copy()
temp_ref_data.layers['DPNMF_IQR'] = norm_temp_ref_data.copy()

In [ ]:
temp_data = temp_data[temp_data.layers['DPNMF_IQR'].max(1)!=0]
temp_ref_data = temp_ref_data[temp_ref_data.layers['DPNMF_IQR'].max(1)!=0]
temp_data.shape,temp_ref_data.shape

In [ ]:
""" Try Harmony Anchor Reference"""
merged_data = anndata.AnnData(X=np.concatenate([temp_data.layers['DPNMF_IQR'].astype(float),temp_ref_data.layers['DPNMF_IQR'].astype(float)]),
                              var=temp_data.var,
                              obs=pd.DataFrame(np.concatenate([np.array(temp_data.obs.index),np.array(temp_ref_data.obs.index)])))
merged_data.obs['dataset'] = np.concatenate([np.array(['a' for i in range(len(temp_data.obs))]),np.array(['b' for i in range(len(temp_ref_data.obs))])])
merged_data.obs['label'] = np.concatenate([np.array(['unknown' for i in range(len(temp_data.obs))]),np.array(temp_ref_data.obsm['Level_3_subclass_label'].astype(str))])
merged_data.obsm['DPNMF'] = merged_data.X.copy()
merged_data.layers['DPNMF'] = merged_data.X.copy()
merged_data.raw = merged_data

import scanpy.external as sce
sce.pp.harmony_integrate(merged_data, 'dataset',basis='DPNMF',adjusted_basis='DPNMF_harmony',reference_values='b')

temp_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'a'].obsm['DPNMF_harmony'])
temp_ref_data.obsm['DPNMF_harmony'] = np.array(merged_data[merged_data.obs['dataset'] == 'b'].obsm['DPNMF_harmony'])

In [ ]:
""" Recluster """
temp_data.X = temp_data.layers['DPNMF_IQR']
sc.tl.pca(temp_data, svd_solver='arpack')
sc.pl.pca(temp_data)
sc.pl.pca_variance_ratio(temp_data, log=False)

In [ ]:
""" Recluster """
sc.pp.neighbors(temp_data, n_neighbors=10, n_pcs=10)
sc.tl.leiden(temp_data)
sc.tl.paga(temp_data)
sc.pl.paga(temp_data, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(temp_data, init_pos='paga')
sc.pl.umap(temp_data, color=['leiden'])
plt.figure(figsize=[25,15])
sc.pl.embedding(temp_data,'stage', color=['leiden'])

In [ ]:
""" Check Correlation with ss Data """
from scipy.stats import pearsonr,spearmanr
temp_ref_data.obs['label'] = temp_ref_data.obsm['Level_3_subclass_label']
ref_cell_types = np.unique(temp_ref_data.obs['label'])
measured_cell_types = np.unique(temp_data.obs['leiden']) 
corr_matrix = np.zeros([ref_cell_types.shape[0],measured_cell_types.shape[0]])
for i,r in enumerate(ref_cell_types):
    r_v = temp_ref_data[temp_ref_data.obs['label']==r].obsm['DPNMF_harmony'].mean(0)
    for j,m in enumerate(measured_cell_types):
        m_v = temp_data[temp_data.obs['leiden']==m].obsm['DPNMF_harmony'].mean(0)
        corr_matrix[i,j] = spearmanr(r_v,m_v)[0]
        # corr_matrix[i,j] = pearsonr(r_v,m_v)[0]
corr_df = pd.DataFrame(corr_matrix,index=ref_cell_types,columns=measured_cell_types)
plt.figure(figsize=[10,10])
sns.clustermap(corr_df,cmap='bwr',center=0,row_cluster=False)
plt.title('Correlation')
plt.show()
""" Use These as label transfer """
converter = {measured_cell_types[i]:j for i,j in enumerate(np.array(corr_df.index)[np.array(corr_df).argmax(0)])}
transfered = np.array([converter[i] for i in temp_data.obs['leiden']])
temp_data.obs['Level_3_subclass_label_predicted'] = transfered
temp_data.obsm['Level_3_subclass_label_predicted'] = transfered
sc.pl.embedding(temp_data,'stage', color=['leiden','Level_3_subclass_label_predicted'])

In [ ]:
temp = pd.DataFrame(data.obs['Level_3_subclass_label_predicted'])
temp.loc[temp_data.obs.index] = np.array(temp_data.obs['Level_3_subclass_label_predicted'])[:,None]
data.obs['Level_3_subclass_label_predicted'] = np.array(temp)
data.obsm['Level_3_subclass_label_predicted'] = np.array(data.obs['Level_3_subclass_label_predicted'])

# Visualization

In [ ]:
reference_data

In [ ]:
data

In [ ]:
np.unique(reference_data.obsm['Level_1_class_label'])

In [ ]:
plt.figure(figsize=[10,10])
plt.bar(2*np.array(range(24)),reference_data[reference_data.obsm['Level_2_neighborhood_label']=='DG_SUB_CA'].layers['DPNMF_IQR'].mean(0),color='r')
plt.bar(2*np.array(range(24))+1,data[data.obsm['Level_2_neighborhood_label_predicted']=='DG_SUB_CA'].layers['DPNMF_IQR'].mean(0),color='k')
plt.show()

In [ ]:
reference_data

In [ ]:
plt.figure(figsize=[10,10])
plt.bar(2*np.array(range(24)),reference_data[reference_data.obsm['Level_1_class_label']=='Glutamatergic'].layers['DPNMF_IQR'].mean(0),color='r')
plt.bar(2*np.array(range(24))+1,data[data.obsm['Level_1_class_label_predicted']=='Glutamatergic'].layers['DPNMF_IQR'].mean(0),color='k')
plt.show()

In [ ]:
reference_data

In [ ]:
reference_data[reference_data.obsm['Level_3_subclass_label']=='V3d'].obsm['Level_1_class_label']

In [ ]:
data[data.obsm['Level_2_neighborhood_label_predicted']=='Astro/Olig_Other'].X.mean(0)